## Playing with RDKit

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions
DrawingOptions.bondLineWidth=1.8

In [ ]:
template = Chem.MolFromSmiles('c1nccc2n1ccc2')
#AllChem.Compute2DCoords(template)
template

In [ ]:
Chem.Draw.IPythonConsole.InstallIPythonRenderer()

In [ ]:
mol = Chem.MolFromSmiles('C=C=C')
mol = AllChem.AddHs(mol)
DrawingOptions.includeAtomNumbers=True
mol

In [ ]:
text = widgets.Text(
    value='',
    placeholder='SMILES input',
    description='String:',
    disabled=False
)
output = widgets.Output()

display(text, output)

def rdkit_2d_struct(smiles):
    m = Chem.MolFromSmiles(smiles)
    return m

def on_text_change(change):
    smiles = change['new']
    try:
        m = rdkit_2d_struct(smiles)
        with output:
            print(m)
    except:
        pass

text.observe(on_text_change, names='value')

In [ ]:
from aiidalab_widgets_base import AiidaNodeViewWidget, ProcessNodesTreeWidget, ProcessMonitor
import ipywidgets as ipw

process_tree = ProcessNodesTreeWidget()
node_view = AiidaNodeViewWidget(layout={"width": "auto", "height": "auto"})
ipw.dlink(
    (process_tree, "selected_nodes"),
    (node_view, "node"),
    transform=lambda nodes: nodes[0] if nodes else None,
)

w = ipw.VBox(children=[process_tree, node_view])
process_monitor = ProcessMonitor(
            timeout=0.1,
            callbacks=[
                process_tree.update,
            ],
)
ipw.dlink((process_tree, "process"), (process_monitor, "process"))
workchain_pk = calc_node.pk
process_tree.process = load_node(workchain_pk)
display(w)

## Storing experimental spectrum in the database

In [ ]:
from aiida.plugins import DataFactory
import numpy as np
import aiida

aiida.load_profile()

XyData = DataFactory('array.xy')
xy = XyData()

In [ ]:
bibliography = ' I. Magneron, R. Thévenet, A. Mellouki, G. Le Bras, G.K. Moortgat, and K. Wirtz, "A study of the photolysis and OH-initiated oxidation of acrolein and trans-crotonaldehyde", J. Phys. Chem. A 106, 2526-2537 (2002); DOI: 10.1021/jp013413a '
source = {'db_name': 'UV/VIS Spectral Atlas',
          'db_uri': 'http://satellite.mpic.de/spectral_atlas',
          'uri': 'http://satellite.mpic.de/spectral_atlas/cross_sections/Organics%20(carbonyls)/Aldehydes(aliphatic)/CH2=CHCHO_Magneron(2002)_298K_282-390nm.txt',
          'id': 'HGINCPLSRVDWNT-UHFFFAOYSA-N', # inChi key
          'version': '2022',
          'description': bibliography,
          'license': '?'
}
energy_nm = np.array([282,284,286,288,290,292,294,296,298,300,302,304,306,308,310,312,314,316,318,320,322,324,326,328,330,332,334,336,338,340,342,344,346,348,350,352,354,356,358,360,362,364,366,368,370,372,374,376,378,380,382,384,386,388,390]) 
y = np.array([8.4e-21,9.7e-21,1.08e-20,1.23e-20,1.46e-20,1.62e-20,1.8e-20,1.97e-20,2.18e-20,2.47e-20,2.7e-20,2.85e-20,3.09e-20,3.29e-20,3.59e-20,3.92e-20,4.15e-20,4.21e-20,4.47e-20,4.65e-20,5.08e-20,5.17e-20,5.34e-20,5.2e-20,5.31e-20,5.44e-20,5.8e-20,6.24e-20,5.46e-20,5.31e-20,5.1e-20,5.12e-20,5.3e-20,5.17e-20,5.94e-20,5.79e-20,4.18e-20,3.63e-20,3.28e-20,3.92e-20,3.72e-20,2.86e-20,3.74e-20,3.82e-20,2.17e-20,1.58e-20,1.14e-20,1.14e-20,1.24e-20,1.1e-20,8.4e-21,7.9e-21,1.18e-20,4.9e-21,2.5e-21])
xy.set_x(energy_nm, 'energy', 'nm')
xy.set_y(y, 'cross section', 'cm^2')

In [ ]:
xy.set_source(source=source)

In [ ]:
xy.set_attribute('smiles', 'C=CC=O')

In [ ]:
xy.base.attributes.all

In [ ]:
node = xy.store()

In [ ]:
node.base.extras.set("smiles", 'C=CC=O')

In [ ]:
from aiida.orm import QueryBuilder
XyData = DataFactory('array.xy')
qb = QueryBuilder()
qb.append(XyData, filters={
    'extras.smiles': 'C=CC=O'
})
qb.all()

## Wigner sampling

In [ ]:
%aiida

In [ ]:
import aiidalab_ispg.workflows.wigner_position as wigner

In [ ]:
output_dict = load_node(pk=1797)

In [ ]:
from ase import Atoms
from aiida.engine import calcfunction
from aiida.orm import StructureData, TrajectoryData

@calcfunction
def sample_wigner(orca_output_dict, nsample):
    frequencies = output_dict['vibfreqs']
    masses = output_dict['atommasses']
    normal_modes = output_dict['vibdisps']
    elements = output_dict['elements']
    min_coord = output_dict['atomcoords'][-1]
    natom = output_dict['natom']
    # convert to Bohrs
    ANG2BOHRS = 1./0.529177211
    coordinates = []
    for iat in range(natom):
        coordinates.append([min_coord[iat][0]*ANG2BOHRS, min_coord[iat][1]*ANG2BOHRS, min_coord[iat][2]*ANG2BOHRS])
    seed = 1235235
    w = wigner.Wigner(elements, masses, coordinates, frequencies, normal_modes, seed)
    
    wigner_list = []
    for i in range(nsample.value):
        wigner_coord = w.get_sample()
        # Convert to angstroms
        wigner_coord_ang = []
        for iat in range(natom):
            wigner_coord_ang.append([wigner_coord[iat][0]/ANG2BOHRS, wigner_coord[iat][1]/ANG2BOHRS, wigner_coord[iat][2]/ANG2BOHRS])
        # TODO: We shouldn't need to specify cell
        # https://github.com/aiidateam/aiida-core/issues/5248
        ase_struct = Atoms(positions=wigner_coord_ang, symbols=elements, cell=(1., 1., 1.))
        wigner_list.append(StructureData(ase=ase_struct))
    return TrajectoryData(structurelist=wigner_list)

In [ ]:
struct = sample_wigner(output_dict, Int(5))

In [ ]:
struct

In [ ]:
coordinates

In [ ]:
seed = 1235235
w = wigner.Wigner(elements, masses, coordinates, frequencies, normal_modes, seed)
coordinates = w.get_sample()
coordinates

In [ ]:
ase_struct.get_positions()

In [ ]:
wigner_struct.store()

In [ ]:
wigner_struct

In [ ]:
import ase
ase_struct = ase.Atoms(symbols="H", positions=[(0.0, 0.0, 0.0)])
ase_struct.cell

In [ ]:
StructureData(ase=ase_struct)